# Part A: Data
# Introduction
This notebook contains all code that has been used to prepare the EStreams and other data for the analysis performed in the "How landscape data quality affects our perception of dominant processes in large-sample hydrology studies?" paper by do Nascimento et al. (in review). To be able to run this notebook, please ensure that you have downloaded the acompanying data of the paper. All links can be found in the data section of the paper.

Author: Thiago Nascimento (thiago.nascimento@eawag.ch)

# Import modules

In [1]:
import pandas as pd
import numpy as np
import os
import geopandas as gpd
import matplotlib.pyplot as plt
import matplotlib as mpl
import warnings
from sklearn.preprocessing import LabelEncoder
warnings.simplefilter(action='ignore', category=Warning)


# Configurations

In [2]:
# Only editable variables:
# Relative path to your local directory
PATH = ".."

# Set the directory:
os.chdir(PATH)

# Import data

### - Network information

In [3]:
network_estreams = pd.read_csv('data/EStreams/streamflow_gauges/estreams_gauging_stations.csv', encoding='utf-8')
network_estreams.set_index("basin_id", inplace = True)

# Convert 'date_column' and 'time_column' to datetime
network_estreams['start_date'] = pd.to_datetime(network_estreams['start_date'])
network_estreams['end_date'] = pd.to_datetime(network_estreams['end_date'])

network_estreams

,gauge_id,gauge_name,gauge_country,gauge_provider,river,lon_snap,lat_snap,lon,lat,elevation,...,num_continuous_days,num_days_gaps,num_days_reliable,num_days_noflag,num_days_suspect,gauge_flag,duplicated_suspect,watershed_group,gauges_upstream,nested_catchments
basin_id,,,,,,,,,,,,,,,,,,,,,
AT000001,200014,Bangs,AT,AT_EHYD,Rhein,9.534835,47.273748,9.534835,47.273748,420,...,9497,0.0,0.0,9497.0,0.0,B,['CH000197'],1,16,"['AT000001', 'CH000010', 'CH000046', 'CH000048..."
AT000002,200048,Schruns (Vonbunweg),AT,AT_EHYD,Litz,9.913677,47.080301,9.913677,47.080301,673,...,23103,0.0,0.0,23103.0,0.0,B,['CH000221'],1,1,['AT000002']
AT000003,231662,Loruens-Aeule,AT,AT_EHYD,Ill,9.847765,47.132821,9.847765,47.132821,579,...,13513,0.0,0.0,13513.0,0.0,B,['CH000215'],1,2,"['AT000002', 'AT000003', 'CH000221']"
AT000004,200592,Kloesterle (OEBB),AT,AT_EHYD,Alfenz,10.061843,47.128994,10.061843,47.128994,1014,...,8765,0.0,0.0,8765.0,0.0,B,['CH000227'],1,1,['AT000004']
AT000005,200097,Buers (Bruecke L82),AT,AT_EHYD,Alvier,9.802668,47.150770,9.802668,47.150770,564,...,10957,0.0,0.0,10957.0,0.0,B,['CH000214'],1,3,"['AT000005', 'CH000214']"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
UAGR0017,6682300,BASHTANOVKA,UA,UA_GRDC,KACHA,33.894739,44.691884,33.900000,44.683333,NaN,...,3652,0.0,0.0,3652.0,0.0,B,NaN,1988,1,['UAGR0017']
UAGR0018,6682500,YALTA,UA,UA_GRDC,DERE-KIOY,34.166667,44.500000,34.166667,44.500000,16,...,3652,0.0,0.0,3652.0,0.0,B,NaN,1989,1,['UAGR0018']
UAGR0019,6683010,PIONERSKOE,UA,UA_GRDC,SALHYR,34.199841,44.887685,34.200000,44.883333,307,...,3652,0.0,0.0,3652.0,0.0,B,NaN,1990,1,['UAGR0019']


In [59]:
network_estreams_filtered_quality_eobs = pd.read_csv('results/network_estreams_filtered_quality_eobs.csv', encoding='utf-8')
network_estreams_filtered_quality_eobs.set_index("basin_id", inplace = True)

network_estreams_filtered_quality_eobs

,gauge_id,gauge_name,gauge_country,gauge_provider,river,lon_snap,lat_snap,lon,lat,elevation,...,num_continuous_days,num_days_gaps,num_days_reliable,num_days_noflag,num_days_suspect,gauge_flag,duplicated_suspect,watershed_group,gauges_upstream,nested_catchments
basin_id,,,,,,,,,,,,,,,,,,,,,
GB000598,77003,Liddel Water at Rowanburnfoot,GB,GB_NRFA,Liddel Water at Rowanburnfoot,-2.920000,55.070000,-2.920000,55.070000,27.1,...,17532,0.0,0.0,17532.0,0.0,B,NaN,1159,1,['GB000598']
DEBY0895,24374805,Würzburg,DE,DE_BY,Pleichach,9.948428,49.799355,9.948428,49.799355,176.10,...,14306,2367.0,0.0,16918.0,0.0,D,NaN,1,1,['DEBY0895']
SE000009,1160,KUKKASJÄRVI,SE,SE_SMHI,SANGISÄLVEN,23.323600,66.111700,23.323600,66.111700,NaN,...,16932,5589.0,30734.0,0.0,90.0,D,NaN,1885,1,['SE000009']
CH000149,2458,Valangin,CH,CH_CAMELS,Seyon,6.900000,47.010000,6.900000,47.010000,665,...,14610,0.0,0.0,14610.0,0.0,B,NaN,1,1,['CH000149']
GB000120,27025,Rother at Woodhouse Mill,GB,GB_NRFA,Rother at Woodhouse Mill,-1.350000,53.370000,-1.350000,53.370000,28.7,...,16710,944.0,0.0,20971.0,0.0,B,NaN,968,2,"['GB000121', 'GB000120']"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
GB000434,53007,Frome (Somerset) at Tellisford,GB,GB_NRFA,Frome (Somerset) at Tellisford,-2.280000,51.310000,-2.280000,51.310000,35.1,...,21945,0.0,0.0,21945.0,0.0,B,NaN,1092,1,['GB000434']
AT000231,206292,Au (Mittlere Au),AT,AT_EHYD,Laudach,13.915166,48.038339,13.915166,48.038340,368,...,15706,0.0,0.0,15706.0,0.0,B,NaN,2,1,['AT000231']
FR002418,V145431002,V1454320,FR,FR_EAUFRANCE,Le Furans à Pugieu [Pont du Martinet],5.642407,45.820879,5.642407,45.820879,248,...,14518,0.0,14223.0,0.0,295.0,A,NaN,27,2,"['FR002417', 'FR002418']"


In [63]:
network_estreams_filtered_quality_eobs = network_estreams_filtered_quality_eobs[network_estreams_filtered_quality_eobs.gauge_country == "CZ"]

network_estreams_filtered_quality_eobs

,gauge_id,gauge_name,gauge_country,gauge_provider,river,lon_snap,lat_snap,lon,lat,elevation,...,num_continuous_days,num_days_gaps,num_days_reliable,num_days_noflag,num_days_suspect,gauge_flag,duplicated_suspect,watershed_group,gauges_upstream,nested_catchments
basin_id,,,,,,,,,,,,,,,,,,,,,
CZ000066,69000,Lázně Bělohrad,CZ,CZ_CHMU,Javorka,15.583472,50.431888,15.583472,50.431888,NaN,...,9053,53.0,0.0,14983.0,0.0,B,NaN,3,1,['CZ000066']
CZ000279,233500,Brenná,CZ,CZ_CHMU,Ploučnice,14.627915,50.651938,14.627915,50.651938,NaN,...,9192,5113.0,0.0,9923.0,0.0,E,NaN,3,5,"['CZ000275', 'CZ000276', 'CZ000277', 'CZ000278..."
CZ000356,303000,Věřňovice,CZ,CZ_CHMU,Olše,18.418995,49.931003,18.418995,49.931003,NaN,...,15036,0.0,0.0,15036.0,0.0,B,NaN,43,20,"['CZ000345', 'CZ000346', 'CZ000347', 'CZ000348..."
CZ000504,456000,Skalní Mlýn,CZ,CZ_CHMU,Punkva,16.719793,49.369542,16.719793,49.369542,NaN,...,15036,0.0,0.0,15036.0,0.0,B,NaN,2,3,"['CZ000501', 'CZ000502', 'CZ000504']"
CZ000455,414500,VD Bojkovice,CZ,CZ_CHMU,Kolelač,17.843778,49.049442,17.843778,49.049442,NaN,...,15036,0.0,0.0,15036.0,0.0,B,NaN,2,2,"['CZ000454', 'CZ000455']"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
CZ000088,92000,Pelešany,CZ,CZ_CHMU,Libuňka,15.170779,50.571139,15.170779,50.571139,NaN,...,14671,0.0,0.0,14671.0,0.0,B,NaN,3,2,"['CZ000087', 'CZ000088']"
CZ000288,241000,Srbská Kamenice,CZ,CZ_CHMU,Kamenice,14.351506,50.814043,14.351506,50.814043,NaN,...,13941,0.0,0.0,13941.0,0.0,B,NaN,3,1,['CZ000288']
CZ000103,109500,Zátoň,CZ,CZ_CHMU,Vltava,14.336846,48.726830,14.336846,48.726830,NaN,...,12114,0.0,0.0,12114.0,0.0,B,NaN,3,5,"['CZ000099', 'CZ000100', 'CZ000101', 'CZ000102..."


In [66]:
list_catchments1 = pd.read_csv('data/catchment_list.txt', encoding='utf-8', header=None)
list_catchments2 = pd.read_csv('data/catchment_list_DK.txt', encoding='utf-8', header=None)

list_catchments = pd.concat([list_catchments1, list_catchments2], ignore_index=True) 


catchment_list = list_catchments.iloc[:, 0].tolist()  # Assuming there's only one column
catchment_list

['AT000002',
 'AT000003',
 'AT000005',
 'AT000007',
 'AT000010',
 'AT000014',
 'AT000015',
 'AT000016',
 'AT000018',
 'AT000019',
 'AT000021',
 'AT000022',
 'AT000023',
 'AT000027',
 'AT000028',
 'AT000029',
 'AT000030',
 'AT000038',
 'AT000039',
 'AT000040',
 'AT000041',
 'AT000042',
 'AT000043',
 'AT000044',
 'AT000048',
 'AT000049',
 'AT000050',
 'AT000051',
 'AT000052',
 'AT000054',
 'AT000060',
 'AT000062',
 'AT000065',
 'AT000067',
 'AT000068',
 'AT000069',
 'AT000071',
 'AT000072',
 'AT000074',
 'AT000075',
 'AT000076',
 'AT000078',
 'AT000080',
 'AT000081',
 'AT000086',
 'AT000087',
 'AT000089',
 'AT000090',
 'AT000093',
 'AT000094',
 'AT000095',
 'AT000097',
 'AT000098',
 'AT000101',
 'AT000103',
 'AT000105',
 'AT000106',
 'AT000107',
 'AT000111',
 'AT000115',
 'AT000116',
 'AT000117',
 'AT000118',
 'AT000122',
 'AT000123',
 'AT000124',
 'AT000125',
 'AT000129',
 'AT000130',
 'AT000132',
 'AT000133',
 'AT000135',
 'AT000137',
 'AT000138',
 'AT000140',
 'AT000141',
 'AT000142',

In [69]:
catchment_list_CZ = network_estreams_filtered_quality_eobs.index.tolist()

catchment_list = catchment_list + catchment_list_CZ

catchment_list

['AT000002',
 'AT000003',
 'AT000005',
 'AT000007',
 'AT000010',
 'AT000014',
 'AT000015',
 'AT000016',
 'AT000018',
 'AT000019',
 'AT000021',
 'AT000022',
 'AT000023',
 'AT000027',
 'AT000028',
 'AT000029',
 'AT000030',
 'AT000038',
 'AT000039',
 'AT000040',
 'AT000041',
 'AT000042',
 'AT000043',
 'AT000044',
 'AT000048',
 'AT000049',
 'AT000050',
 'AT000051',
 'AT000052',
 'AT000054',
 'AT000060',
 'AT000062',
 'AT000065',
 'AT000067',
 'AT000068',
 'AT000069',
 'AT000071',
 'AT000072',
 'AT000074',
 'AT000075',
 'AT000076',
 'AT000078',
 'AT000080',
 'AT000081',
 'AT000086',
 'AT000087',
 'AT000089',
 'AT000090',
 'AT000093',
 'AT000094',
 'AT000095',
 'AT000097',
 'AT000098',
 'AT000101',
 'AT000103',
 'AT000105',
 'AT000106',
 'AT000107',
 'AT000111',
 'AT000115',
 'AT000116',
 'AT000117',
 'AT000118',
 'AT000122',
 'AT000123',
 'AT000124',
 'AT000125',
 'AT000129',
 'AT000130',
 'AT000132',
 'AT000133',
 'AT000135',
 'AT000137',
 'AT000138',
 'AT000140',
 'AT000141',
 'AT000142',

In [70]:
network_estreams_filtered = network_estreams.loc[catchment_list]
network_estreams_filtered

,gauge_id,gauge_name,gauge_country,gauge_provider,river,lon_snap,lat_snap,lon,lat,elevation,...,num_continuous_days,num_days_gaps,num_days_reliable,num_days_noflag,num_days_suspect,gauge_flag,duplicated_suspect,watershed_group,gauges_upstream,nested_catchments
basin_id,,,,,,,,,,,,,,,,,,,,,
AT000002,200048,Schruns (Vonbunweg),AT,AT_EHYD,Litz,9.913677,47.080301,9.913677,47.080301,673,...,23103,0.0,0.0,23103.0,0.0,B,['CH000221'],1,1,['AT000002']
AT000003,231662,Loruens-Aeule,AT,AT_EHYD,Ill,9.847765,47.132821,9.847765,47.132821,579,...,13513,0.0,0.0,13513.0,0.0,B,['CH000215'],1,2,"['AT000002', 'AT000003', 'CH000221']"
AT000005,200097,Buers (Bruecke L82),AT,AT_EHYD,Alvier,9.802668,47.150770,9.802668,47.150770,564,...,10957,0.0,0.0,10957.0,0.0,B,['CH000214'],1,3,"['AT000005', 'CH000214']"
AT000007,231688,Beschling,AT,AT_EHYD,Ill,9.678850,47.200301,9.678850,47.200301,485,...,13514,0.0,0.0,13514.0,0.0,B,['CH000205'],1,8,"['AT000002', 'AT000003', 'AT000004', 'AT000005..."
AT000010,200154,Laterns,AT,AT_EHYD,Frutz,9.728853,47.256933,9.728853,47.256933,831,...,10517,3.0,0.0,24104.0,0.0,B,['CH000209'],1,1,['AT000010']
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
CZ000088,92000,Pelešany,CZ,CZ_CHMU,Libuňka,15.170779,50.571139,15.170779,50.571139,NaN,...,14671,0.0,0.0,14671.0,0.0,B,NaN,3,2,"['CZ000087', 'CZ000088']"
CZ000288,241000,Srbská Kamenice,CZ,CZ_CHMU,Kamenice,14.351506,50.814043,14.351506,50.814043,NaN,...,13941,0.0,0.0,13941.0,0.0,B,NaN,3,1,['CZ000288']
CZ000103,109500,Zátoň,CZ,CZ_CHMU,Vltava,14.336846,48.726830,14.336846,48.726830,NaN,...,12114,0.0,0.0,12114.0,0.0,B,NaN,3,5,"['CZ000099', 'CZ000100', 'CZ000101', 'CZ000102..."


In [71]:
network_estreams_filtered.groupby('gauge_country').size()

gauge_country
AT    397
CH    157
CZ    350
DE    976
DK    168
ES    231
FR    500
GB    538
IS      4
SE     25
dtype: int64

In [35]:
import tqdm as tqdm 
path_estreams = r'C:\Users\nascimth\Documents\Thiago\Eawag\Python\Data\data'

In [37]:
import pandas as pd
import os
import shutil
import zipfile

In [ ]:
for basin_id in tqdm.tqdm(network_estreams_filtered.index):
    data = pd.read_csv(path_estreams+"\estreams_timeseries_"+basin_id+".csv", index_col=0)

    #data["q_mean"] = timeseries_estreams[basin_id]
    #data.to_csv(r"C:\Users\nascimth\Documents\Thiago\Eawag\Python\Data\data"+"\estreams_timeseries_"+basin_id+".csv")

    output_folder = f"results\hbv\{basin_id}"  # Folder named after the basin_id
    txt_filename = os.path.join(output_folder, "ptq_estreams.txt")
    txt_filename_evap = os.path.join(output_folder, "evap_estreams.txt")

    zip_filename = f"results\hbv\{basin_id}.zip"

    # Create the output folder
    os.makedirs(output_folder, exist_ok=True)

    # Ensure the required columns exist
    required_columns = ["p_mean", "t_mean", "q_mean"]
    df_selected = data[required_columns].copy()

    df_selected.index = pd.to_datetime(df_selected.index)

    df_selected = df_selected.loc["01-10-1995":"30-09-2015"]

    # Format the date index to YYYYMMDD
    df_selected.insert(0, "date", df_selected.index.strftime("%Y%m%d"))

    df_selected.columns = ["Date", "P", "T", "Q"]

    # Save the formatted data to ptq_estreams.txt (tab-separated, no header)
    df_selected.to_csv(txt_filename, sep="\t", header=True, index=False)

    df_pet = data["pet_mean"].copy()
    df_pet.index = pd.to_datetime(df_pet.index)
    df_pet = df_pet.loc["01-10-1995":"30-09-2015"]
    # Convert Series to DataFrame and rename column
    df_pet = df_pet.to_frame(name=f"basin.{basin_id}")    
    df_pet.to_csv(txt_filename_evap, sep="\t", header=True, index=False)


    # Create a zip archive of the folder
    with zipfile.ZipFile(zip_filename, "w", zipfile.ZIP_DEFLATED) as zipf:
        for root, _, files in os.walk(output_folder):
            for file in files:
                file_path = os.path.join(root, file)
                zipf.write(file_path, os.path.relpath(file_path, output_folder))

    # Remove the uncompressed folder after zipping (optional)
    shutil.rmtree(output_folder)

    print(f"Compressed folder '{zip_filename}' successfully created!")




df_selected

  0%|          | 2/3346 [00:00<08:13,  6.77it/s]

Compressed folder 'results\hbv\AT000002.zip' successfully created!
Compressed folder 'results\hbv\AT000003.zip' successfully created!


  0%|          | 4/3346 [00:00<07:59,  6.97it/s]

Compressed folder 'results\hbv\AT000005.zip' successfully created!
Compressed folder 'results\hbv\AT000007.zip' successfully created!


  0%|          | 6/3346 [00:00<07:51,  7.08it/s]

...

 10%|█         | 345/3346 [00:57<08:14,  6.07it/s]

Compressed folder 'results\hbv\AT000513.zip' successfully created!
Compressed folder 'results\hbv\AT000514.zip' successfully created!


 10%|█         | 347/3346 [00:57<07:56,  6.29it/s]

Compressed folder 'results\hbv\AT000515.zip' successfully created!
Compressed folder 'results\hbv\AT000518.zip' successfully created!


 10%|█         | 348/3346 [00:57<07:45,  6.44it/s]

# End